In [5]:
import loader_helper
import os
import numpy as np
import tqdm
import nibabel as nii
from scipy.ndimage import zoom

In [6]:
data_path = '../../../data/MICCAI_BraTS_2019_Data_Training/ALL'

In [7]:
files = [f for f in os.listdir(data_path) if os.path.isdir(os.path.join(data_path,f))]
files.sort()

print(len(files))

335


In [8]:
data, annotation, affine = loader_helper.read_multimodal(data_path, files[0])

In [9]:
def get_bbox(data):
    bboxes = np.stack([loader_helper.bbox3(d) for d in data],axis=0)
    return np.stack([np.min(bboxes[:,0],axis=0),np.max(bboxes[:,1],axis=0)],axis=0)

In [10]:
bboxes = []
affines = []
for f in tqdm.tqdm(files):
    data, annotation, affine = loader_helper.read_multimodal(data_path, f, False)
    affines.append(affine)
    bboxes.append(get_bbox(data))
bboxes = np.stack(bboxes,axis=0)
print(bboxes.shape)

100%|██████████| 335/335 [01:26<00:00,  3.87it/s]

(335, 2, 3)


In [11]:
sizes = [b[1] - b[0] for b in bboxes]
print(np.max(sizes,axis=0))

[156 188 148]


In [12]:
crop_size = [loader_helper.closest_to_k(i,16) for i in np.max(sizes,axis=0)]
print(crop_size)

[160, 192, 160]


In [15]:
min = np.array([0,0,0])
max = np.array([240, 240, 155])

output_dir = '../../../data/2019_cropped'

crop_size = np.array(crop_size)

#resample_to_size = (64,64,64)
#scale_factor = np.array(crop_size) / np.array(resample_to_size)

for idx, f in enumerate(tqdm.tqdm(files)):
    data, annotation, affine = loader_helper.read_multimodal(data_path, f, True)
    b = get_bbox(data)
    size = b[1] - b[0]
    
    output = np.zeros(shape=(data.shape[0],)+tuple(crop_size))
    out_annotation = np.zeros(shape=tuple(crop_size))
    
    diff = np.array(crop_size) - np.array(size)
    low = diff // 2
    high = low - diff
    bbox = b - np.stack([low,high])

    index_input_min = np.maximum(bbox[0],min)
    index_input_max = np.minimum(bbox[1],max)
    
    size = index_input_max - index_input_min
    

    index_output_min = crop_size//2 - size//2 
    index_output_max = crop_size//2 + size - size//2

    
    output[:,index_output_min[0]:index_output_max[0],index_output_min[1]:index_output_max[1],index_output_min[2]:index_output_max[2]] =\
    data[:,index_input_min[0]:index_input_max[0],index_input_min[1]:index_input_max[1],index_input_min[2]:index_input_max[2]]
    
    out_annotation[index_output_min[0]:index_output_max[0],index_output_min[1]:index_output_max[1],index_output_min[2]:index_output_max[2]] =\
    annotation[0,index_input_min[0]:index_input_max[0],index_input_min[1]:index_input_max[1],index_input_min[2]:index_input_max[2]]
    
    

    suffixes = ['_t1.nii.gz','_t1ce.nii.gz','_t2.nii.gz','_flair.nii.gz']
    
    os.makedirs(name=os.path.join(output_dir,f), exist_ok=True)
    
    #affine = np.diag([affine[i,i]*scale_factor[i] for i in range(3)]+[1])    
    
    for jdx, d in enumerate(output):
        #d = zoom(d, 1/scale_factor, order=3, mode='constant', cval=0)
    
        output_header = nii.Nifti1Image(d.astype(np.float32), affine)
        nii.save(output_header, os.path.join(output_dir,f,f+suffixes[jdx]))
        
    
    #out_annotation = zoom(out_annotation, 1/scale_factor, order=0, mode='constant', cval=0)
    out_annotation[out_annotation==4] = 3
    output_header = nii.Nifti1Image(out_annotation.astype(np.uint8), affine)
    nii.save(output_header, os.path.join(output_dir,f,f+'_seg.nii.gz'))

100%|██████████| 335/335 [04:58<00:00,  1.09it/s]
